## Using RapidAPI to scrape Zillow data

### Key attributes needs for analysis are Zestimate, price per square ft and rentZestimate

We are using Georgia (GA) Listing data to get all the addresses of properties which are on interest. We then use following sources to augment that data:
* Zillow
* IRS (Tax data)
* FHFA (HPI data)

The data is then further transformed by feature extraction or feature engineeting work, for example:
* Schools (from Zillow)
* Lot Size (from Zillow and GA Listings)

In [1]:
#!pip install googlesearch-python

In [9]:
#pip install google

     |████████████████████████████████| 45 kB 472 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Importing Key Libraries

In [145]:
from googlesearch import search # get zpid
import pandas as pd
import numpy as np
import requests
import json
import time
import io
import plotly.express as px

# show all columns
pd.set_option('display.max_columns', None)

### <font color=blue> This function use google search and find zpid (primary key for zillow)
#### RapidAPI requires zpid

In [2]:
def get_zpid(street=None, city=None, state=None, zip_code=None, full_address=None):
  # get search query string
  if full_address == None:
    try:
      query = '{0}, {1}, {2} {3} zillow home details'.format(street, city, state, str(zip_code))
    except:
      return 'Please enter a query string or address details'
  else:
    query = full_address + ' zillow home details'

  # get google search results
  search_results = search(query, tld='com', lang='en', num=3, start=0, stop=1, pause=0)
  search_results_list = [u for u in search_results]
  url = search_results_list[0] # extract first returned result
  
  # return zpid
  try:
    return [x for x in url.split('/') if 'zpid' in x][0].split('_')[0]
  except:
    return None

### <font color=blue> Register and test the Rapid API key

In [3]:
rapid_api_key = "please_use_a_valid_key"

In [4]:
def get_property_detail(rapid_api_key, zpid):
  # get property details from API
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid} # zpid

  headers = {
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
    "X-RapidAPI-Key": rapid_api_key # your key here
  }

  # request data
  return requests.request("GET", url, headers=headers, params=querystring)


Testing the API

In [5]:
import requests

url = "https://zillow-com1.p.rapidapi.com/property"

querystring = {"zpid":"2080998890"}

headers = {
    "X-RapidAPI-Key": "rapid_api_key",
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"listingProvider":{"enhancedVideoURL":null,"showNoContactInfoMessage":false,"postingGroupName":null,"isZRMSourceText":null,"showLogos":null,"disclaimerText":null,"sourceText":null,"title":"Listing Provided by","logos":{},"postingWebsiteURL":null,"agentLicenseNumber":null,"postingWebsiteLinkText":"See listing website","enhancedDescriptionText":null,"agentName":null},"buildingPermits":null,"propertyTaxRate":1.01,"contact_recipients":[{"agent_reason":1,"zpro":null,"recent_sales":0,"review_count":56,"display_name":"Jim Pullin","zuid":"X1-ZU11t0lr5j8rqx5_34dhh","rating_average":5,"badge_type":"Premier Agent","phone":{"prefix":"207","areacode":"206","number":"8763"},"image_url":"https:\/\/photos.zillowstatic.com\/h_n\/ISfkgeu1boe63o1000000000.jpg"}],"solarPotential":null,"longitude":-122.34212,"countyFIPS":null,"cityId":16037,"timeOnZillow":"611 days","url":"\/homedetails\/757-Hayes-St-Seattle-WA-98109\/2080998890_zpid\/","zestimate":null,"imgSrc":"https:\/\/photos.zillowstatic.com\/fp\/306

In [6]:
# property address
property_address = "84 Whippoorwill Circle, Kingsland, GA 31548" # https://www.zillow.com/homedetails/11622-Pure-Pebble-Dr-Riverview-FL-33569/66718658_zpid/

# search query
query = property_address + ' zillow home details'
print('Search this phrase in Google Search:', query)

Search this phrase in Google Search: 84 Whippoorwill Circle, Kingsland, GA 31548 zillow home details


In [7]:
# google search results
search_results = search(query, tld='com', lang='en', num=3, start=0, stop=3, pause=0)
search_results_list = [u for u in search_results] # get all results
search_results_list

['https://www.zillow.com/homedetails/84-Whippoorwill-Cir-Kingsland-GA-31548/76375189_zpid/',
 'https://www.compass.com/listing/84-whippoorwill-circle-kingsland-ga-31548/1020322606183510697/',
 'https://www.coldwellbankerhomes.com/ga/kingsland/84-whippoorwill-circle/pid_46488818/']

In [8]:
# get the first search result
url = search_results_list[0] # extract first returned result
url

'https://www.zillow.com/homedetails/84-Whippoorwill-Cir-Kingsland-GA-31548/76375189_zpid/'

In [9]:
# extract the zpid
zpid = [x for x in url.split('/') if 'zpid' in x][0].split('_')[0]
print('Zpid of the property is:', zpid )

Zpid of the property is: 76375189


In [10]:
# get property details from API
url = "https://zillow-com1.p.rapidapi.com/property"

querystring = {"zpid":zpid} # zpid

headers = {
    "X-RapidAPI-Host": "zillow-com1.p.rapidapi.com",
    "X-RapidAPI-Key": rapid_api_key # your key here
}

# request data
response = requests.request("GET", url, headers=headers, params=querystring)
# show success
response.status_code # 200 is success!

200

In [11]:
response.json()

{'listingProvider': None,
 'buildingPermits': None,
 'propertyTaxRate': 0.98,
 'contact_recipients': [{'agent_reason': 1,
   'zpro': None,
   'recent_sales': 1,
   'review_count': 66,
   'display_name': 'Hannah Melton',
   'zuid': 'X1-ZUzlemig891155_8qndh',
   'rating_average': 5,
   'badge_type': 'Premier Agent',
   'phone': {'prefix': '380', 'areacode': '912', 'number': '3440'},
   'image_url': 'https://photos.zillowstatic.com/h_n/ISfo72urg9xp1t1000000000.jpg'}],
 'solarPotential': {'sunScore': 64.35,
  'solarFactor': 3.04,
  'buildFactor': 52,
  'climateFactor': 6.67,
  'electricityFactor': 2.64},
 'longitude': -81.66023,
 'countyFIPS': '13039',
 'cityId': 52779,
 'timeOnZillow': '146 days',
 'url': '/homedetails/84-Whippoorwill-Cir-Kingsland-GA-31548/76375189_zpid/',
 'zestimate': 235700,
 'imgSrc': 'https://photos.zillowstatic.com/fp/9d94004ba25c8463f8d134d7a4e7f3fc-p_d.jpg',
 'zpid': 76375189,
 'zipcode': '31548',
 'livingAreaValue': 1618,
 'zestimateLowPercent': None,
 'isListed

In [12]:
# transform data to pandas dataframe
df_property_detail = pd.json_normalize(data=response.json())
print('Num of rows:', len(df_property_detail))
print('Num of cols:', len(df_property_detail.columns))
df_property_detail.head()

Num of rows: 1
Num of cols: 257


,listingProvider,buildingPermits,propertyTaxRate,contact_recipients,longitude,countyFIPS,cityId,timeOnZillow,url,zestimate,imgSrc,zpid,zipcode,livingAreaValue,zestimateLowPercent,isListedByOwner,propertyTypeDimension,streetAddress,county,taxHistory,stateId,countyId,timeZone,homeType,livingAreaUnits,comingSoonOnMarketDate,livingArea,bathrooms,annualHomeownersInsurance,state,rentZestimate,building,brokerId,yearBuilt,brokerageName,dateSold,price,pageViewCount,description,homeStatus,homeFacts,latitude,datePosted,bedrooms,nearbyHomes,monthlyHoaFee,priceHistory,favoriteCount,schools,zestimateHighPercent,mlsid,city,providerListingID,country,currency,contingentListingType,solarPotential.sunScore,solarPotential.solarFactor,solarPotential.buildFactor,solarPotential.climateFactor,solarPotential.electricityFactor,resoFacts.hasAttachedProperty,resoFacts.frontageType,resoFacts.poolFeatures,resoFacts.flooring,resoFacts.foundationDetails,resoFacts.accessibilityFeatures,resoFacts.hasGarage,resoFacts.hasPetsAllowed,resoFacts.bodyType,resoFacts.topography,resoFacts.landLeaseExpirationDate,resoFacts.hasAdditionalParcels,resoFacts.waterViewYN,resoFacts.atAGlanceFacts,resoFacts.offerReviewDate,resoFacts.aboveGradeFinishedArea,resoFacts.buyerAgencyCompensationType,resoFacts.carportSpaces,resoFacts.cityRegion,resoFacts.mainLevelBathrooms,resoFacts.hasPrivatePool,resoFacts.associationFeeIncludes,resoFacts.waterSource,resoFacts.coveredSpaces,resoFacts.inclusions,resoFacts.hasFireplace,resoFacts.zoningDescription,resoFacts.garageSpaces,resoFacts.sewer,resoFacts.elevationUnits,resoFacts.exclusions,resoFacts.mainLevelBedrooms,resoFacts.numberOfUnitsVacant,resoFacts.hasWaterfrontView,resoFacts.listAOR,resoFacts.lotSize,resoFacts.entryLevel,resoFacts.irrigationWaterRightsAcres,resoFacts.greenWaterConservation,resoFacts.stories,resoFacts.livingArea,resoFacts.numberOfUnitsInCommunity,resoFacts.listingTerms,resoFacts.otherStructures,resoFacts.associationFee,resoFacts.marketingType,resoFacts.greenIndoorAirQuality,resoFacts.greenSustainability,resoFacts.hasHomeWarranty,resoFacts.associationPhone,resoFacts.greenBuildingVerificationType,resoFacts.elementarySchool,resoFacts.bedrooms,resoFacts.architecturalStyle,resoFacts.listingId,resoFacts.structureType,resoFacts.interiorFeatures,resoFacts.horseAmenities,resoFacts.electric,resoFacts.developmentStatus,resoFacts.lotFeatures,resoFacts.roofType,resoFacts.waterSources,resoFacts.constructionMaterials,resoFacts.fireplaceFeatures,resoFacts.doorFeatures,resoFacts.appliances,resoFacts.daysOnZillow,resoFacts.bathroomsPartial,resoFacts.bathroomsHalf,resoFacts.depositsAndFees,resoFacts.canRaiseHorses,resoFacts.buildingName,resoFacts.hasAssociation,resoFacts.petsMaxWeight,resoFacts.commonWalls,resoFacts.totalActualRent,resoFacts.additionalParcelsDescription,resoFacts.associationName,resoFacts.allowedPets,resoFacts.communityFeatures,resoFacts.tenantPays,resoFacts.middleOrJuniorSchool,resoFacts.lotSizeDimensions,resoFacts.roadSurfaceType,resoFacts.hasOpenParking,resoFacts.yearBuilt,resoFacts.livingAreaRangeUnits,resoFacts.irrigationWaterRightsYN,resoFacts.bathroomsFull,resoFacts.horseYN,resoFacts.incomeIncludes,resoFacts.highSchool,resoFacts.leaseTerm,resoFacts.elevation,resoFacts.cropsIncludedYN,resoFacts.attic,resoFacts.subdivisionName,resoFacts.windowFeatures,resoFacts.ownership,resoFacts.woodedArea,resoFacts.middleOrJuniorSchoolDistrict,resoFacts.associationPhone2,resoFacts.buyerAgencyCompensation,resoFacts.heating,resoFacts.frontageLength,resoFacts.virtualTourURLUnbranded,resoFacts.associationAmenities,resoFacts.propertySubType,resoFacts.hasCooling,resoFacts.belowGradeFinishedArea,resoFacts.hasElectricOnProperty,resoFacts.foundationArea,resoFacts.zoning,resoFacts.hoaFee,resoFacts.livingAreaRange,resoFacts.basementYN,resoFacts.availabilityDate,resoFacts.waterBodyName,resoFacts.waterfrontFeatures,resoFacts.specialListingConditions,resoFacts.buildingAreaSource,resoFacts.taxAnnualAmount,resoFacts.parking,resoFacts.hasView,resoFa

In [13]:
# retrieve property detail elements
bedrooms = df_property_detail['bedrooms'].iloc[0]
bathrooms = df_property_detail['bathrooms'].iloc[0]
year_built = df_property_detail['yearBuilt'].iloc[0]
property_type = df_property_detail['homeType'].iloc[0]
living_area = df_property_detail['resoFacts.livingArea'].iloc[0]
lot_size = df_property_detail['resoFacts.lotSize'].iloc[0]
lot_dimensions = df_property_detail['resoFacts.lotSizeDimensions'].iloc[0]
zoning = df_property_detail['resoFacts.zoning'].iloc[0]
livingAreaValue = df_property_detail['livingAreaValue'].iloc[0]
schools = df_property_detail['schools'].iloc[0]
# estimates
zestimate = df_property_detail['zestimate'].iloc[0]
rent_zestimate = df_property_detail['rentZestimate'].iloc[0]

print('PROPERTY DETAILS FOR: ', property_address)
print('------------------------------------------------------------------')
print('  Bedrooms: {}'.format( bedrooms))
print('  Bathrooms: {}'.format( bathrooms))
print('  Year Built: {}'.format( year_built))
print('  Living Area: {}'.format( living_area))
print('  Lot Size: {}'.format( lot_size))
print('  Lot Dimensions: {}'.format( lot_dimensions))
print('  Zoning: {}'.format( zoning))
print('  livingAreaValue: {}'.format( livingAreaValue))
print('  schools: {}'.format( schools))
print('  Property Type: {}'.format( property_type))
print('  Zestimate:  {}'.format(zestimate))
print('  Rent Zestimate: {}'.format(rent_zestimate))

PROPERTY DETAILS FOR:  84 Whippoorwill Circle, Kingsland, GA 31548
------------------------------------------------------------------
  Bedrooms: 3
  Bathrooms: 2
  Year Built: 1986
  Living Area: 1,618 sqft
  Lot Size: None
  Lot Dimensions: None
  Zoning: None
  livingAreaValue: 1618
  schools: [{'link': 'https://www.greatschools.org/school?id=00276&state=GA', 'rating': 8, 'totalCount': 1, 'distance': 0.9, 'assigned': None, 'name': 'David L Rainer Elementary School', 'studentsPerTeacher': 16, 'isAssigned': True, 'size': 456, 'level': 'Primary', 'grades': 'PK-5', 'type': 'Public'}, {'link': 'https://www.greatschools.org/school?id=00278&state=GA', 'rating': 6, 'totalCount': 1, 'distance': 2.2, 'assigned': None, 'name': 'Camden Middle School', 'studentsPerTeacher': 16, 'isAssigned': True, 'size': 1046, 'level': 'Middle', 'grades': '6-8', 'type': 'Public'}, {'link': 'https://www.greatschools.org/school?id=00275&state=GA', 'rating': 9, 'totalCount': 1, 'distance': 3.6, 'assigned': None, '

### <font color=blue> The Short Look Up csv file created from data provided by Haystack, but consists of only 4 columns

In [89]:
# read file
df_upload = pd.read_csv('short_lookup.csv')
print('Num of rows:', len(df_upload))
df_upload.head()

Num of rows: 5146


,address,city,state,zip
0,101 College Street,Kingsland,Georgia,31548
1,169 School House Creek Drive,Saint Marys,Georgia,31558
2,1621 Sandpiper Court,Saint Marys,Georgia,31558
3,346 Laurel Landing Boulevard,Kingsland,Georgia,31548
4,81 Ossabaw Way,Waverly,Georgia,31565


### <font color=blue> This is a one time data pull script, do not run again!

In [19]:
# create empty list to store api response for property details
df_list = []
zpid_list = []

# iterate through dataframe
for index, row in df_upload.iterrows():
  # get property address attributes by column name
  street = row['address']
  city = row['city']
  state = row['state']
  zip_code = row['zip'] 

  # get zpid
  zpid = get_zpid(street=street, city=city, state=state, zip_code=zip_code)
  zpid_list.append(zpid)

  # wait 1 second to not overload Google Search
  time.sleep(1.5)

  # get property details
  # catch errors (possibly no zpid)
  try:
    response = get_property_detail(rapid_api_key, zpid)
    _df = pd.json_normalize(data=response.json())
    # append data to list of property details
    df_list.append(_df)
  except:
    print('Unable to retrieve data for : {0}, {1}, {2}, {3}'.format(street, city, state, zip_code))

Unable to retrieve data for : 4342 White Surrey Drive NW, Kennesaw, Georgia, 30144


In [21]:
# join property details data
df_prop_detail = pd.concat(df_list)
df_prop_detail.head()

In [251]:
len(df_prop_detail.zpid.unique())

4609

### <font color=blue> We are saving the zillow data with 200+ attributes, which will save us from re-running the Rapid API code

In [ ]:
#Save data to a file for manual inspection
df_prop_detail.to_csv('full_output.csv', index=False)

In [ ]:
import json
with open("zpid_list", "w") as fp:
     json.dump(zpid_list, fp)